In [1]:
import pysal as ps
import numpy as np

In [2]:
df = ps.pdio.read_files(ps.examples.get_path('columbus.shp'))[['geometry', 'INC', 'HOVAL', 'CRIME']]

This used to look like:

<img src='./old_repr.png'>

now it's:

In [3]:
df.head()

,geometry,INC,HOVAL,CRIME
0,"POLYGON ((8.6241292953491 14.2369804382324, 8....",19.531,80.467003,15.725980
1,"POLYGON ((8.2527904510498 14.2369403839111, 8....",21.232,44.567001,18.801754
2,"POLYGON ((8.6533050537109 14.0080900192261, 8....",15.956,26.350000,30.626781
3,"POLYGON ((8.4594993591309 13.8203496932983, 8....",4.477,33.200001,32.387760
4,"POLYGON ((8.6852741241455 13.6395196914673, 8....",11.252,23.225000,50.731510


This can get read/written using default pandas plaintext tools

In [4]:
df.to_csv('./test.csv', index=False)

In [5]:
import pandas as pd

In [6]:
df2 = pd.read_csv('./test.csv')

In [7]:
df2.head()

,geometry,INC,HOVAL,CRIME
0,"POLYGON ((8.6241292953491 14.2369804382324, 8....",19.531,80.467003,15.725980
1,"POLYGON ((8.2527904510498 14.2369403839111, 8....",21.232,44.567001,18.801754
2,"POLYGON ((8.6533050537109 14.0080900192261, 8....",15.956,26.350000,30.626781
3,"POLYGON ((8.4594993591309 13.8203496932983, 8....",4.477,33.200001,32.387760
4,"POLYGON ((8.6852741241455 13.6395196914673, 8....",11.252,23.225000,50.731510


In [8]:
type(df2.geometry[0]), df2.geometry[0]

(str,
 'POLYGON ((8.6241292953491 14.2369804382324, 8.5597000122070 14.7424497604370, 8.8094520568848 14.7344303131104, 8.8084125518799 14.6365203857422, 8.9193048477173 14.6385002136230, 9.0871381759644 14.6304903030396, 9.0999650955200 14.2448301315308, 9.0150470733643 14.2418403625488, 9.0089511871338 13.9950599670410, 8.8181400299072 14.0020503997803, 8.6533050537109 14.0080900192261, 8.6429023742676 14.0897102355957, 8.6325922012329 14.1705904006958, 8.6258258819580 14.2236700057983, 8.6241292953491 14.2369804382324))')

But, it won't automatically convert if so. To do this, we need to actually tell `pandas` to parse the WKT column. This isn't that hard, but I will smooth this out:

In [9]:
parser = ps.core.IOHandlers.wkt.WKTParser().fromWKT
df2['geometry'] = df2.geometry.apply(ps.core.IOHandlers.wkt.WKTParser().fromWKT)

In [10]:
type(df2.geometry[0]), df2.geometry[0]

(pysal.cg.shapes.Polygon,
 POLYGON ((8.6241292953491 14.2369804382324, 8.5597000122070 14.7424497604370, ...)

In [11]:
np.testing.assert_allclose(df2.geometry[0].vertices, df.geometry[0].vertices)

Still to do on this:

0. Repeated use of WKT causes coordinate drift, especially if decisions about how things are truncated change. So, if possible, we should try to use wkb, meaning...
1. I need to implement `__reduce__` methods for binary serialization.
2. Implement `to_dataframe` from standard pysal filehandlers
3. Add multipolygon to wkt parsing.

In [ ]:
ps.open('columbus.dbf').to_dataframe(); ps.pdio.read_files()

In [14]:
ps.cg.Chain(df.geometry[0].vertices)ma

LINESTRING (8.62413 14.23698, 8.55970 14.74245, ...

In [16]:
df.geometry[0]._precision = 5

In [17]:
df.geometry[0]

POLYGON ((8.62413 14.23698, 8.55970 14.74245, ...

In [18]:
mps = ps.open(ps.examples.get_path('Polygon.shp')).read()

In [19]:
mps[0]

MULTIPOLYGON (((-0.0108093977040866 -0.2582597347495280, -0.0074876647089110 -0.2549380017543524, ...

In [20]:
mps[1]

POLYGON ((0.053964395701836 -0.312653112545527, 0.051473095955455 -0.352513908487634, ...